# 1.从html提取信息写入csv

In [ ]:
# 调用外部的py
from demo import *
hello()

# 2.将空缺的日期补齐

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
df=pd.read_csv('../csv绘图/info.csv',header=0,sep=',') #filename可以直接从盘符开始，标明每一级的文件夹直到csv文件，header=None表示头部为空，sep=' '表示数据间使用空格作为分隔符，如果分隔符是逗号，只需换成 ‘，’即可。
# 2.按日期进行升序排列
df2 = df.sort_values(by='day')
# 3.提取出 day列  将其转化为 yyyy/mm/dd的格式  将 str=2017/1/27 23:06  转化为  datetime类型
d1 = [datetime.strptime(x, '%Y/%m/%d %H:%M') for x in list(df2["day"])]
#       将 datetime 转化为 str 2017/01/27
d_2017 = [datetime.strftime(x, '%Y/%m/%d') for x in list(d1)]
# 4.创建全年的日子
all_2017=[datetime.strftime(x,'%Y/%m/%d') for x in list(pd.date_range(start='2017/01/01',  end="2017/12/31"))]
# 5.将上两项做差求出没有记录的日子
absent_2017 = [i for i in all_2017 if i not in d_2017]

# 6.将没有记录的日子进行补充  [day:当天,title:空,longitude,latitude,num:0]
dataframe = pd.DataFrame({'day': absent_2017,
                            'title': [None]*len(absent_2017),
                            'longitude': [None]*len(absent_2017),
                            'latitude': [None]*len(absent_2017),
                            'num': [0]*len(absent_2017)})
dataframe.to_csv("./info.csv",index=False, sep=',',mode='a', header=False)


In [ ]:
absent_2017 = ['2017/07/10','2017/07/12','2017/07/11']
absent_2017_np = [time.mktime(time.strptime(x, '%Y/%m/%d')) for x in absent_2017]

In [ ]:
from datetime import datetime
import time       
timeArray = time.strptime('2017/7/11 00:00', '%Y/%m/%d %H:%M')
#转换成时间戳
time.mktime(timeArray)     

In [ ]:
absent_2017_np

# 3.处理重复的日子

## 3.1 手动修改日期 并检查查看

In [78]:
import pandas as pd
import numpy as np
from datetime import datetime

# First：找到没有进行创作的日子
# 1.读取文件 第一行为列名称
res_path = './data/我的抗战2.0_2020_info.csv'
# res_path = './data/2020_info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 手动修改 日期  
# 截取day的date————2017/02/02  是否唯一进行检查
M = [x.split(" ")[0] for x in df["day"].tolist()]
# 给原 df加一列 date
df['day_date'] = M
# 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
day_date = df[df['day_date'].duplicated(keep=False)]
M = day_date[['day','title','day_date']]
print(len(M))
M

47


,day,title,day_date
8,2020/01/09 12:18,3.339 你漏了底，又没有勇气和决心，结局很难乐观。,2020/01/09
9,2020/01/09 22:56,3.340 2h 侯副两过弱招不值一提 心疼老爹 指挥老爹和我统一战线,2020/01/09
44,2020/02/13 12:39,4.23 8h 查手机格式化 切尔诺贝利禁区,2020/02/13
45,2020/02/13 12:40,4.24 计算机视觉,2020/02/13
46,2020/02/13 23:15,4.25 10h 更换学习资源 发现超好用的jupyter 因为学习受阻中午睡得不踏实...,2020/02/13
108,2020/04/15 07:24,2020.03月总结,2020/04/15
109,2020/04/15 22:34,4.87 7h 开始GANs 狗静静外出溜达永乐大师的 高斯十七边形解法神奇 一战时的...,2020/04/15
161,2020/06/06 16:24,2020.5总结,2020/06/06
162,2020/06/06 22:30,4.138 休息写总结 跑步5公里打底理发 华莱士全鸡 幸福终点站 狗静静回...,2020/06/06
170,2020/06/14 23:08,4.145 3h 挣扎沟通商量休假 跑步,2020/06/14


## 3.2自动整理日期
2019/08/19 12:00,,,,0,1566187200.0
2019/08/20 15:02,3.196  转文全部被拒  心情失落  又开始考虑复原,0,0,28,1566284520.0
2019/08/20 15:04,3.197  第一天便无法做到自律 

In [ ]:
def edit_info(res_path):
    # 手动修改 日期  
    # 截取day的date————2017/02/02 12:26  -->  2017/02/02  是否唯一进行检查
    df=pd.read_csv(res_path,header=0,sep=',') 
    M = [x.split(" ")[0] for x in df["day"].tolist()]
    # 给原 df加一列 date
    df['day_date'] = M
    # 得到date重复的行  保留出现第一次的值  duplicated(keep=False)
    day_date = df[df['day_date'].duplicated(keep=False)]
    M = day_date[['day','title','day_date']]
    print("重复的天数为：",len(M))
   
    all_index = M.index.tolist() 
      # 剔除 0 元素
    if 0 in all_index:
        all_index.remove(0)
    # 2.判断 index-1 是否为系统自动生成的 (num值是否为0)
    del_index = []

    for x in all_index:
        if ((df[x:x+1].day_date == '2020/02/13').bool()):
            print("2020/02/13")
        if ((df[x-1:x].num == 0).bool()):
            print("删除行",df[x-1:x].day)
            del_index.append(x-1)
            # df = df.drop([x-1])
        # 3.是——删掉 index-1这一行 
        # 4.修改 df[index] 的 date-1
            temp = df[x:x+1]
            dt = datetime.datetime.strptime(temp.at[temp.index.values[0],'day'],'%Y/%m/%d %H:%M')
            out_date = (dt + datetime.timedelta(days=-1)).strftime('%Y/%m/%d %H:%M')
            temp.day[x] = out_date
    # 5.更新csv文件
    df = df.drop(del_index)
    df.to_csv(res_path,index=False, sep=',')

## 3.3将文件按年进行分表

In [73]:
# 1.读取文件
res_path = '../info.csv'
df=pd.read_csv(res_path,header=0,sep=',') 
# 2.按照day_date(2018/01/01)  列的前四位  2018进行分表
# day_date_list = list(df['day_date'])
day_date_list = [x.split('/')[0] for x in list(df['day_date'])]
year_list = list(set(day_date_list))
year_list = [eval(x) for x in list(set(day_date_list))]
# 3.将年份由小到大进行排序
year_list.sort()
# 4.分割年份  年份第一次出现的位置进行
all_years = [df[day_date_list.index(str(year_list[x])):day_date_list.index(str(year_list[x]+1))] for x in range(len(year_list)-1)]
# 最后一年
all_years.append(df[day_date_list.index(str(year_list[-1])):])
# 3.输出表
[df_year.to_csv("./data/"+str(year)+"_info.csv",index=False, sep=',')  for (year,df_year) in zip(year_list,all_years)]

[None, None, None, None]